### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('./data/movies_clean.csv')
reviews = pd.read_csv('./data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [6]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat) ) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for n in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i,j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k]  += learning_rate*2*error*movie_mat[k,j]
                        movie_mat[k,j] += learning_rate*2*error*user_mat[i,k]
                    

        # print results
        print('%d \t\t %f' % (n+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [7]:
user_mat, movie_mat = FunkSVD(ratings_mat, learning_rate=0.0005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 55.304698
2 		 52.834120
3 		 50.203327
4 		 47.421630
5 		 44.505502
6 		 41.478756
7 		 38.372262
8 		 35.223076
9 		 32.072980
10 		 28.966454


In [8]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[4.82845285 4.56343263 4.97716162 5.20832409]
 [2.65187457 2.48994398 2.75186251 2.87601008]
 [2.08898664 2.05070974 2.04856253 2.37581583]
 [3.74170279 3.41860552 3.61894309 4.09282698]
 [3.0454788  3.11552472 3.1913728  3.35525562]
 [3.07495492 2.72547101 3.02533944 3.23190348]
 [3.55073692 3.37687026 3.63158459 3.62230907]
 [4.76198934 4.57971457 4.78997128 5.63835513]
 [3.52400276 3.48042809 3.51821408 3.92451446]
 [3.27700871 3.20665172 3.27913471 3.63464655]
 [2.54016493 2.3553745  2.53268469 2.67606732]
 [2.15339834 2.07256079 2.19711142 2.37602777]
 [3.89934588 3.79431505 4.17210577 4.50645909]
 [2.29859604 2.00690392 2.30691929 2.66789339]
 [3.99652454 3.77514604 4.06517334 4.69116379]
 [3.84072256 3.62401366 3.9926191  4.0448351 ]
 [4.17693159 4.02491212 4.05107563 4.72533828]
 [3.81775077 3.41491655 3.72069332 3.87103279]
 [3.53696555 3.524438   3.73482474 3.99712004]
 [4.16398147 3.93221724 4.30037008 4.6164549 ]]
[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [9]:
user_mat, movie_mat =FunkSVD(ratings_mat, learning_rate=0.0005, iters=250)# use your function with 4 latent features, lr of 0.005 and 10 iterations #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 60.387093
2 		 58.059924
3 		 55.590706
4 		 52.974083
5 		 50.211366
6 		 47.311163
7 		 44.289800
8 		 41.171363
9 		 37.987238
10 		 34.775062
11 		 31.577068
12 		 28.437882
13 		 25.401955
14 		 22.510864
15 		 19.800800
16 		 17.300513
17 		 15.029968
18 		 12.999815
19 		 11.211682
20 		 9.659171
21 		 8.329351
22 		 7.204508
23 		 6.263930
24 		 5.485533
25 		 4.847217
26 		 4.327890
27 		 3.908162
28 		 3.570740
29 		 3.300584
30 		 3.084887
31 		 2.912933
32 		 2.775890
33 		 2.666565
34 		 2.579167
35 		 2.509065
36 		 2.452589
37 		 2.406841
38 		 2.369543
39 		 2.338912
40 		 2.313552
41 		 2.292372
42 		 2.274519
43 		 2.259324
44 		 2.246265
45 		 2.234928
46 		 2.224989
47 		 2.216190
48 		 2.208325
49 		 2.201230
50 		 2.194772
51 		 2.188845
52 		 2.183361
53 		 2.178248
54 		 2.173448
55 		 2.168912
56 		 2.164599
57 		 2.160476
58 		 2.156512
59 		 2.152685
60 		 2.148974
61 		 2.145361
62 		 2.141831
63 

In [10]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.05894794  9.13817173 10.82238309  9.81156758]
 [ 9.40815351  5.39784033  8.64333843  9.77147722]
 [ 7.23775638  8.53653549  9.24723876  6.91558028]
 [ 9.62728991  7.49177676 10.06521151  9.75868978]
 [ 9.28264519  5.77166459  8.59656257  9.46913787]
 [ 6.57618433  5.79976716  7.44704611  6.60613384]
 [ 8.66806337  8.04265229 10.39459515  8.87605012]
 [ 8.68550218  6.21511922  8.50184356  8.77365094]
 [ 8.04193032  7.79666511  9.23791444  7.96727202]
 [ 8.16487055  6.29787994  7.71842082  8.06356211]
 [ 8.66319287  8.33068808  9.63944689  8.43019755]
 [ 9.32874644  9.53007618 10.27790504  8.78821512]
 [ 9.36651708  8.43762977  9.9263708   9.16724215]
 [ 6.14173329  6.23754221  7.25177506  6.01420453]
 [ 9.95177037  8.16301733 10.11919835  9.78834859]
 [ 9.62030342  8.74882038 10.22528542  9.37927883]
 [ 8.48021851  7.51475053  8.69074491  8.18916984]
 [ 8.94264187  5.6802578   5.65012729  8.09830513]
 [ 7.78645858  6.34747999  8.89990215  8.17019385]
 [ 8.03952917  7.75940413  8.68

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [11]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [12]:
# run SVD on the matrix with the missing value
user_mat, movie_mat =FunkSVD(ratings_mat, learning_rate=0.0005, iters=250)# use your function with 4 latent features, lr of 0.005 and 10 iterations #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 57.582583
2 		 55.009807
3 		 52.316424
4 		 49.503475
5 		 46.579267
6 		 43.559496
7 		 40.467043
8 		 37.331322
9 		 34.187115
10 		 31.072927
11 		 28.028917
12 		 25.094578
13 		 22.306391
14 		 19.695697
15 		 17.287018
16 		 15.097005
17 		 13.134094
18 		 11.398838
19 		 9.884824
20 		 8.579979
21 		 7.468089
22 		 6.530323
23 		 5.746631
24 		 5.096913
25 		 4.561906
26 		 4.123798
27 		 3.766587
28 		 3.476242
29 		 3.240700
30 		 3.049763
31 		 2.894934
32 		 2.769208
33 		 2.666872
34 		 2.583299
35 		 2.514766
36 		 2.458291
37 		 2.411493
38 		 2.372477
39 		 2.339734
40 		 2.312063
41 		 2.288508
42 		 2.268306
43 		 2.250849
44 		 2.235645
45 		 2.222301
46 		 2.210498
47 		 2.199977
48 		 2.190528
49 		 2.181976
50 		 2.174180
51 		 2.167020
52 		 2.160398
53 		 2.154232
54 		 2.148452
55 		 2.143000
56 		 2.137827
57 		 2.132890
58 		 2.128154
59 		 2.123588
60 		 2.119165
61 		 2.114862
62 		 2.110661
63 	

In [13]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.457104505743104:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [14]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, learning_rate=0.0005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.026053
2 		 35.289887
3 		 29.551384
4 		 25.005426
5 		 21.582697
6 		 19.030198
7 		 17.086568
8 		 15.559490
9 		 14.323816
10 		 13.299461


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [ ]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users) )# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")